In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import json
import time
from random import sample
import ipfshttpclient
from os import listdir
from os.path import isfile, join
import re
import praw

import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

from substrate_helpers import *

In [3]:
reddit_creds = json.load(open(".reddit_creds.json", "r"))
reddit = praw.Reddit(
    client_id=reddit_creds["client_id"],
    client_secret=reddit_creds["client_secret"],
    password=reddit_creds["password"],
    user_agent=reddit_creds["user_agent"],
    username=reddit_creds["username"],
)

In [15]:
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)
bob = Keypair.create_from_uri('//Bob')
bob_msa_id = get_msa_id(bob)

In [7]:
r_all = reddit.subreddit('all')

In [8]:
client = ipfshttpclient.connect()

In [11]:
path = "/tmp/"

In [97]:
def mint_reddit_users_msa_ids(post, delegate):
    ## USER ##
    try:
        username = post.author.name
        profile_pic = post.author.icon_img
    except:
        username = "removed"
        profile_pic = "removed"

    password = 'password'
    user_wallet = Keypair.create_from_uri('//' + username + password)
    create_msa_with_delegator(delegate, user_wallet, wait_for_inclusion=False, wait_for_finalization=False)
    
    return {"username": username, "password": password, "profile_pic": profile_pic, "user_wallet": user_wallet}

In [112]:
def mint_reddit_users_msa_ids_for_posts(posts, delegate):
    users = {}
    posts_and_comments = posts
    for i, (post_name, post) in enumerate(posts.items()):
        users[post_name] = {post_name: mint_reddit_users_msa_ids(post, delegate)}
        
        top_comments = []
        for comment in post.comments.list()[:10]:
            if comment.parent_id == post.name:
                top_comments.append(comment.name)
                users[post_name][comment.name] = mint_reddit_users_msa_ids(post, delegate)
        for comment in post.comments.list():
            if type(c) != praw.models.reddit.more.MoreComments and c.parent_id in top_comments:
                users[post_name][comment.name] = mint_reddit_users_msa_ids(post, delegate)
        
    return users

In [128]:
def mint_comment(post_data_hash, parent_hash, user, comment):      
    user_wallet = Keypair.create_from_uri('//' + user['username'] + user['password'])
    user_msa_id = get_msa_id(user_wallet)

    comment_data = {
        "post_hash": post_data_hash,
        "parent_hash": parent_hash,
        "depth": comment.depth,
        "body": comment.body,
    }

    comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, comment_schemaId, path+'comments/', wait_for_inclusion=False, wait_for_finalization=False)

    ## comment votes ##
    receipt_ups = mint_votes(user_msa_id, comment.ups, comment_data_hash, post_data_hash, 'comment')
    receipt_downs = mint_votes(user_msa_id, comment.downs, comment_data_hash, post_data_hash, 'comment')
    
    return comment_data_hash

In [132]:
def mint_reddit_posts_and_users(posts, users, delegate):
    for i, (post_name, post) in enumerate(posts.items()):
        user = users[post.name][post.name]
        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }
        user_wallet = Keypair.create_from_uri('//' + user['username'] + user['password'])
        user_msa_id = get_msa_id(user_wallet)
        users[post.name]['user_msa_id'] = user_msa_id
        if user_msa_id is None:
            return False
        
        receipt_user = mint_user(user_msa_id, user['username'], user['password'], user['profile_pic'], user_wallet)

        post_data_hash, receipt_post = mint_data(post_data, user_msa_id, post_schemaId, path+'posts/', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)

        ## post votes ##
        receipt_ups = mint_votes(user_msa_id, post.ups, post_data_hash, post_data_hash, 'post', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)
        receipt_downs = mint_votes(user_msa_id, post.downs, post_data_hash, post_data_hash, 'post', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)
        
        top_comments = {}
        for comment in post.comments.list()[:10]:
            if comment.parent_id == post.name:  
                try:
                    user = users[post.name][comment.name]     
                except:
                    continue
                comment_data_hash = mint_comment(post_data_hash, post_data_hash, user, comment)
                top_comments[comment.name] = comment_data_hash
                
        for comment in post.comments.list():
            if type(comment) != praw.models.reddit.more.MoreComments and comment.parent_id in top_comments:
                try:    
                    user = users[post.name][comment.name]    
                except:
                    continue
                comment_data_hash = mint_comment(post_data_hash, top_comments[comment.parent_id], user, comment)
        
    return True

#         ## FOLLOWS ##
#         users_list = list(users.keys())
#         for follow_user in sample(users_list, min(10, len(users_list))):
#             follow_user(user_msa_id, follow_msa_id)

#         users.append(user_msa_id)

In [113]:
all_users = {}
all_posts = {}

In [114]:
posts = {p.name: p for p in r_all.top(time_filter='day') if p.name not in all_posts}
all_posts.update(posts)

In [115]:
all_users.update(mint_reddit_users_msa_ids_for_posts(posts, bob))

Extrinsic '0x03b73a9e5a5c86210dd2d0e1611fcac7ca796d5cdbe2303365193d506d10aca5' sent and included in block 'None'
Extrinsic '0x3c2f2c8d70996d65772990db58a3786ad8aa707a7d4ba9ecdbe92ed2b653c24c' sent and included in block 'None'
Extrinsic '0x64ed3ca586fd903ec553e4eb66c3ae572d8d46ae1c835d90b5714f86b1fe3bb5' sent and included in block 'None'
Extrinsic '0xbe9c13a2d28df9ea047671300699e7a4c2f589e192249861acd5dc07726603d7' sent and included in block 'None'
Extrinsic '0x9ee56626df2cb97dcf79fd614d9ffacbf9f66e4e08bc2dae6b49c6cc10ec3d2b' sent and included in block 'None'
Extrinsic '0xe5acafd6f14938aea721eeff12d686ae7f0a8fb8f9050b681e95037b6de5b8bf' sent and included in block 'None'
Extrinsic '0x80c2bcbfcbeeaf7763b2d2e5cd13a7d069c5f84c220cf12ca482dde5d9551119' sent and included in block 'None'
Extrinsic '0x96512d85e62699e99508311b3c116a1868394d9f14e480ee970a163e448740ff' sent and included in block 'None'
Extrinsic '0xdbc250bb167f9cba2eced0210a742705102258b4476076ac44992c22f01e5c68' sent and included

In [133]:
result = mint_reddit_posts_and_users(posts, all_users, bob)

Extrinsic '0xb5e6eb38ea83f5bd6b534495e941a8f71e00f7dfee4b1546694e52efdd53c3d9' sent and included in block 'None'
Extrinsic '0x230f15e93ca1f0e7f7702e4bae8a62d18988a99018822d9a169d2289ce6b0706' sent and included in block 'None'
Extrinsic '0xf9f7f340b2234eea47dea73eb870c041850b878223b39e36214bc20d8eaae0e7' sent and included in block 'None'
Extrinsic '0x57bd9935307144371e6754b2ba54fd15e8b053a30b0f730c3e8a9b0e41bd03fb' sent and included in block 'None'
Extrinsic '0xe25e8385bb4a5ba4751bb59129ee56b44b108a00e8921204d72517b89c5622ac' sent and included in block 'None'
Extrinsic '0x9bf225fffbdf600660e6b65b078f41496f93e2c753b56298c142cd5233c14d3c' sent and included in block 'None'
Extrinsic '0xf79f8ab9eff8ee3bc67fb2601c8768d4707304241477df3574c21bed1062ae69' sent and included in block 'None'
Extrinsic '0x6b235e65e3dabec27b8b66b90df0e6f2300d2ab0794f921c8bcc95edd39d5e81' sent and included in block 'None'
Extrinsic '0x6e47c0de580b92fa834cfad1ad5ef06331d759f61dc04275b8d512ce2f264a0e' sent and included

Extrinsic '0x4d00a2f72bd0f60f360f9668a6255456d1bdeeeb367ce57ea36e1f3bc96c33a1' sent and included in block 'None'
Extrinsic '0x6bdfeac2c1bb27a8346553fecb5cb2b5b58218a89f094153cd841f225e4d83be' sent and included in block 'None'
Extrinsic '0xe2d37a91ea1afe9b6f50189bdab6add379db1d5c7fb709386241a7e351cebbee' sent and included in block 'None'
Extrinsic '0xfe4d8153c178d7b4b081f83c498329e00fa9701f7e7275e76d0bbcdf3133b08b' sent and included in block 'None'
Extrinsic '0x8b3d747632d5fb199c3aa29cae03f89fa00c1eca3e1aa7b0c158adfc77211381' sent and included in block 'None'
Extrinsic '0xc035fe4a489677d15b756627ec11a2b357f7549d3f60c623d4fc49da5a9ec72a' sent and included in block 'None'
Extrinsic '0x3c7e0734f636971d7ea572fc0a03ebe8a66bc08255fa6c68bab867ae1865efb8' sent and included in block 'None'
Extrinsic '0x3c80ac1e4a77257400722e07c496da89030a26deafa8db31434835f8832bdd05' sent and included in block 'None'
Extrinsic '0x43443852eb71a610da54dcb9e0fc2363bef8e135a06c57569dbaed428c779e32' sent and included

Extrinsic '0xde2ab66263aaac8eb515a8742ab2f4e1bbb90dd0037dbbe25a0981d6eee1e6c2' sent and included in block 'None'
Extrinsic '0x5179f343b6ffc57fcf07d24470626fd6ef751794dd43d3e92ae4058df7074793' sent and included in block 'None'
Extrinsic '0xa8ec99518fb38b36449b0746951e8781bbe6dcea8c3e838551d2d2b56db0c5f8' sent and included in block 'None'
Extrinsic '0x30a8332244d04db61c3ff12bba3b3506152d6286a8391d80ab54e9d2c255b6b3' sent and included in block 'None'
Extrinsic '0x1031f34396c08f27a6f4a3b7507d7c7a8290a14b57aa044d6755c3d3a91d8d3f' sent and included in block 'None'
Extrinsic '0x74aa44ce4e8bce6e41c909b127f014832abdd5f67b92b07ba5b2c3632fe3664e' sent and included in block 'None'
Extrinsic '0x3fc8af1b2fa4e52380641c8676c7a07e3a0d7236fba7b754363869457b49823c' sent and included in block 'None'
Extrinsic '0x290053c5f7e617954ba00e4d13e4d3b6a0451566b91deb7fa957bb2ffe03e37a' sent and included in block 'None'
Extrinsic '0x067da27f56c9bd83d244493c3b25b82ca92169baaf0a8f7ce79ef1588a980fb7' sent and included

Extrinsic '0x47b8230cf9b93c857f54129e40c2bd03b4c0fb162aaf6231f2e709f957a69d3b' sent and included in block 'None'
Extrinsic '0x26078323f337f7dfb451cc732d2714bafeeefbca7a9867611431307bb38cd69d' sent and included in block 'None'
Extrinsic '0x4c6698640694f27679e9a3e2a63eb2bf1f68873bab3ee91adb709ece6501c1e7' sent and included in block 'None'
Extrinsic '0x972ab2c9af4ff22c0e7e943135c207ff60e878435b07dfa44088fcd729ecf68e' sent and included in block 'None'
Extrinsic '0x2f58c73dbf9e69285bf0d61c29287dd79233136f7b67cff069b3af80c7944fe5' sent and included in block 'None'
Extrinsic '0x8d4145e05a26b11be15551ebd5377a605abb07a8aed8331bd03a6480a00f4864' sent and included in block 'None'
Extrinsic '0x305d8ec3dbcd61a666b31340430fdfb4d5108a7cee157d3aa8a821c6c2125aa1' sent and included in block 'None'
Extrinsic '0x2be1ecb4b16925ebfafafff27ca0cb50b08059115ade6b6068908dc9a978650e' sent and included in block 'None'
Extrinsic '0xaf904c23333bf837dd67bbb7f863f48547cd2a35ebb858390851e1d32a56a21c' sent and included

Extrinsic '0x279bfb7d602fee1918f0ebf25f5bfb2129138f9a54903449d5c36685b0b030a3' sent and included in block 'None'
Extrinsic '0x92c906d3b11d337c829f7f950e81523c38074f309e68b2d04730de4fc0c8aea0' sent and included in block 'None'
Extrinsic '0x04a768ab5dfdd9d88537eff09dc32299734e2464325badca5e4cd62f9932b14f' sent and included in block 'None'
Extrinsic '0xfe5888c4f5c70440a992da8df09f72854e45fca0162a5fe8a7b939d96d89a8e7' sent and included in block 'None'
Extrinsic '0xa5fb8976fd3e049764873e91700c8f59d628b508c7292e26567b9e4b2a4c759b' sent and included in block 'None'
Extrinsic '0x5053a30e8a836e35061171b1edc7edb22b2c3fddf4fd088ed67ae7ab658176bb' sent and included in block 'None'
Extrinsic '0x7ea6ee066d4e0816a39bfd7e752ba9c050eda7126652a8c6b4e5190c62cbae0d' sent and included in block 'None'
Extrinsic '0x83e28a24b7b1bdf8bf8437571879685ce0b2a41afe1646536e33d7d13d04f613' sent and included in block 'None'
Extrinsic '0xf0c220250d06b15cf06beb62f0c62d5736888487714a36e587fe23e8dc19d6f5' sent and included

...

Extrinsic '0xaaba2c394b103f6c6b127b05e41c4538fbcf349ba70d81070f5eeaa04bacfe10' sent and included in block 'None'
Extrinsic '0xc1d31fec3c6eec641b7aac4136b0c62d0a60381b21ef7f824c689babbd44370a' sent and included in block 'None'
Extrinsic '0x55b0d40e2e4698261bdefca480543ce9f2e30f6efbd95067ef5512da4c7475b6' sent and included in block 'None'
Extrinsic '0x1136139458e12b5d13053f84b43fb39f30292058c089a083d8dab69f8c2b6208' sent and included in block 'None'
Extrinsic '0x7828c31256dea7fef350469339326993a9ac30761ca6f66e8975d94e0fbf59d4' sent and included in block 'None'
Extrinsic '0x3b8d02a03e277dc9e68dcd0f4b5f6aa1cf911478d6aed70d81bbfd2c861811a7' sent and included in block 'None'
Extrinsic '0xa2ae53425795a9f01ca6c0686c5e6c9fe5b7073e47de1341fa541c3a0c22ead4' sent and included in block 'None'
Extrinsic '0x542032d144c9e377d1fb1d2d2946b306b0b28faa73dc100f41f7c82d06ebe2a6' sent and included in block 'None'
Extrinsic '0xa68a214c90214d5e483d4f4e167b09e70023dbe5a8374bcdf31417465b9d06a7' sent and included

Extrinsic '0xb444dc3ff06e3208df8691489b638569ab1d09c347b96c8e971332e25c96accd' sent and included in block 'None'
Extrinsic '0x80cb1343a930c356aac282ad0e0c4d25be159b633c0d5be9c7af52dfa3fb8541' sent and included in block 'None'
Extrinsic '0xfdf8db12794c419483107ca9022419e5bcb48de7272b9a4b86d05e2849348377' sent and included in block 'None'
Extrinsic '0xb985de9bb8cbce57bc0fb7a201be7a79548d0935f4191a6085c2cfad9e838c44' sent and included in block 'None'
Extrinsic '0x6c28034e24daa21bee55df45486a20ba635b0960d84237905da43e1b16106293' sent and included in block 'None'
Extrinsic '0x79206941e5f6d42b5c8a9a79194bd75571250294567cf5ef452671382df99248' sent and included in block 'None'
Extrinsic '0x178b4a4c5ef41188f30f62f8fbf0f5a981c7939e1d06f8e4b880c7cd0ecdb8d8' sent and included in block 'None'
Extrinsic '0x192e999224e1590a29447941086c6ec329db1e9ffa481fa1238c17cb61d5df56' sent and included in block 'None'
Extrinsic '0x37debf80037eed9b6a140cff67406c889cd4107a1f87b2a7c1e34b93d0a512d0' sent and included

Extrinsic '0xab80e1b03b776d663f5e001e877928693a89dfe404941a26e96adfcd338eecda' sent and included in block 'None'
Extrinsic '0x5334c40598ad88e7727e0c49cae089683665e23ad3c3e5992b60fa0bdb94efef' sent and included in block 'None'
Extrinsic '0xd5f5450afe30c7cf7d507a7e08a8201fc4a5777c7d2a1767ffa75d332408e6d9' sent and included in block 'None'
Extrinsic '0xfc43e0db4a26661f525715967f64dd2858ab59eda77a98f765a1bd4391d62fc2' sent and included in block 'None'
Extrinsic '0x3cc79846778bcc55e98c7f9249fab25f0487a0f327bd99d97d6939cd0417aa12' sent and included in block 'None'
Extrinsic '0xc5be6985b61f2fe1fe67d76a22f5bf2710e2e017f72ef49cd249855184b56dda' sent and included in block 'None'
Extrinsic '0x7dc3750ee64763fef7f1bdcad098afdf24c9e9934736ecbfe07f98de7940c805' sent and included in block 'None'
Extrinsic '0x5ef100e07c441886074c4fdf29cf4d6f6d2aa57f1b47460e09637babdff70149' sent and included in block 'None'
Extrinsic '0xff7dca371285cd2f4bbb77f1c742a475864156c14b1e5449cda5673eab8305ce' sent and included

Extrinsic '0xad9724a5246989ab76f4f85d01937b65b8dc1c0d05a03eedd674a3f17ce9bd6d' sent and included in block 'None'
Extrinsic '0x203d6b7138863ae06480eed7d87adffe5c8463d0dd976c60aa5fb6c70695c7af' sent and included in block 'None'
Extrinsic '0xe839815de9de9d6cb2f99cf1ae680e5b3b8ddd51d30357b2768dd74212d3fee9' sent and included in block 'None'
Extrinsic '0x8fee38c85b3035cc8fea0d797525e25022fc5a9edb1ff947a1fe8ae50228e85c' sent and included in block 'None'
Extrinsic '0xc25198b58fcb5484ab696b2763c1398915fb303802c5a6f44daae02949f877ed' sent and included in block 'None'
Extrinsic '0x8b77b30e7110ee053a7c9a6bcdc93ab9611b81b88c56ce056e1ec0d7690eff83' sent and included in block 'None'
Extrinsic '0xd7d2c6f2042cbb505c93c64078901ccedd074d0e2cf9a5bf95702cf8bb36ff4f' sent and included in block 'None'
Extrinsic '0x3c69197a0ca08b78553cb69c378c129decddb510ee5f98504208da309583f04d' sent and included in block 'None'
Extrinsic '0x941019a2af239e76a6e7f77e649c86d5ae41ec7d8678464202c35fe57c70e017' sent and included

Extrinsic '0xc18f857e136303a39cabe37038c5fb38551334657a15799fe04e47112aaf13d1' sent and included in block 'None'
Extrinsic '0x4e4ae1e4cd83e80555e4c84b8e4ff5654b45f5933739fd11723cae9b890d4a46' sent and included in block 'None'
Extrinsic '0xdf111cd3728c0ea6e4ef81b392389d17edb7a1c872816f51c9389ea4555acb91' sent and included in block 'None'
Extrinsic '0x333ed04c24b154cb827e377af6798f08999bfdd9bbe7bd007df45715b89c4465' sent and included in block 'None'
Extrinsic '0x3456ada18fb55780fbd652b10b0c3b7e332dfe91893ca793d3aaa67776e2a70e' sent and included in block 'None'
Extrinsic '0x9b09281a1f7681d0c6d0d1e3defbab4f7575ce66b06ffc7a35a34173743e5cdd' sent and included in block 'None'
Extrinsic '0x8c9a80aa959602b10760bab9728c6e7582a2f30f5c930e85a08fe40eaacf9ae5' sent and included in block 'None'
Extrinsic '0x427fc6a240b122b90fde4e25ace7b158868b6d19e4cca751017333ec839f4d54' sent and included in block 'None'
Extrinsic '0x702c41ca3739f6cd7998bdf5cbcf827aee6378d26f6ec389bda8d0a1e3bb0a94' sent and included

In [ ]:
result

# Old

In [20]:
def mint_comments(comments, post, post_data_hash):
    id_to_hash = {}
    for comment in comments:
        try:
            username = comment.author.name
            profile_pic = comment.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        comment_user_msa_id, receipt_comment_user = mint_user(username, password, profile_pic, user_wallet)
            
        if comment.parent_id == post.name:
            parent_hash = post_data_hash
        else:
            parent_hash = id_to_hash[comment.parent_id]
            
        comment_data = {
            "post_hash": post_data_hash,
            "parent_hash": parent_hash,
            "depth": comment.depth,
            "body": comment.body,
        }
        
        comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, path+'comments/' wait_for_inclusion=False, wait_for_finalization=False)
        
        receipt_comment = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
        print(comment_data_hash)
        
        ## comment votes ##
        receipt_ups = mint_votes(comment_user_msa_id, comment.ups, comment_data_hash, post_data_hash, 'comment')
        receipt_downs = mint_votes(comment_user_msa_id, comment.downs, comment_data_hash, post_data_hash, 'comment')
        print(comment.ups, comment.downs)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(comment_user_msa_id, follow_msa_id)
            
        users.append(comment_user_msa_id)

In [23]:
while (True):
    posts = [p for p in r_all.top(time_filter='day')]
    for i, post in enumerate(posts):
        ## USER ##
        try:
            username = post.author.name
            profile_pic = post.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        user_msa_id, receipt_user = mint_user(username, password, profile_pic, user_wallet)

        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "username": username,
            "profile_pic": profile_pic,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }

        post_data_hash, receipt_post = make_post(post_data, user_msa_id, path+'posts/', wait_for_inclusion=False, wait_for_finalization=False)
        print(post_data_hash)

        ## post votes ##
        receipt_ups = mint_votes(user_msa_id, post.ups, post_data_hash, post_data_hash, 'post')
        receipt_downs = mint_votes(user_msa_id, post.downs, post_data_hash, post_data_hash, 'post')
        print(post.ups, post.downs)

        ## COMMENT ##
        mint_comments(post.comments.list()[:10], post, post_data_hash)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(user_msa_id, follow_msa_id)
            
        users.append(user_msa_id)
        
    print("waiting...")
    print()
    # sleep for a bit and then check again
    time.sleep(60*10)

Extrinsic '0xb5cafbb6c313f9261676610e390287c69a7b5739da89ae3b9b74386b113f3b62' sent and included in block 'None'
Extrinsic '0xd002df048664209bfd609f391cb13630d6e06006e77df33c439828fa94cc6db0' sent and included in block 'None'
Extrinsic '0x81bbb6952a69cc25cd539f35c975d31e026e3967e272587811bd9080eaab3ddc' sent and included in block 'None'
Extrinsic '0x68a1a0f067c6c36b3436a3db68774160c18392c5e29dcbfacdccde8fb72bbf20' sent and included in block 'None'
Extrinsic '0xb2f171be69c428488f4f4cad95adca69e2e3c5d822099675be711574e3e2d492' sent and included in block 'None'
Extrinsic '0xb3005c43b597882b171a7f5002a1bdf25a0ac11f111b28e14d3a419859cfd72c' sent and included in block 'None'
Extrinsic '0x65152b49fea7e38f6d34395618bffd635521d17f7bd5a46cdff1ad7c3111db1a' sent and included in block 'None'
Extrinsic '0x45f43a311919562d99b43feee8b6262edc6624fc97145e68168220b77b58a8f8' sent and included in block 'None'
Extrinsic '0x239052065565e66de58466c9b57cee27683e3c98fbf70220e41b0a4e1d94f1f8' sent and included

TypeError: 'NoneType' object is not subscriptable

In [39]:
b = reddit.redditor('abowlofspaghetti')

In [40]:
b.link_karma, b.comment_karma

(7003, 737, 0, 0)

In [47]:
b.submissions.new()

In [49]:
for t in b.submissions.new():
    break

In [52]:
t.selftext

"I'm not a huge fan of GBTC for Bitcoin but it's the only way to get exposure to crypto in my 401k. So I will be dumping all my GBTC whenever an ETF comes available. However, I feel like many people could do that and it'll tank the stock. Is that a possibility? I know it's supposed to represent btc value but it's constantly off by huge percentages."

In [32]:
[t for t in b.awardee_karma.top()]

[]

In [543]:
substrate.get_block()['header']['number']

9694

In [489]:
start_block = 5490

In [42]:
post_pattern = re.compile(f".*,.*,type,parent,data")

In [43]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

post_schemas = {}
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if post_pattern.match(schemaTemp.value):
        print(schemaTemp.value)
        post_schemas[schemaTemp.value] = i

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data


In [48]:
%%time
content_jsons = {}
for schema, schemaId in post_schemas.items():
    params = [
        schemaId,
        {
            "page_size": 10000,
            "from_block": 0,
            "to_block": 10000,
            "from_index": 0,
        }
    ]

    content = substrate.rpc_request(
        method='messages_getBySchema',
        params=params,
    )
    print(schema, content)
    if len(content['result']['content']) > 0:
        content_jsons[schema] = content['result']['content']

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data {'jsonrpc': '2.0', 'result': {'content': [{'block_number': 34, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}, {'block_number': 43, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}], 'has_next': False}, 'id': 139}
CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 41.8 ms


In [45]:
content_jsons

{}

In [353]:
def add_comment(_data, _dict, _is_vote):
    for hsh in _dict.keys():
        if hsh == data[2]:
            if is_vote:
                if '-' in data[3]:
                    # downvote
                    _dict['data']['downvotes'] += data[3]
                else:
                    # upvote
                    _dict['data']['upvotes'] += data[3]
            else:
                _dict[data[3]] = {
                    'data': json.loads(client.cat(_data).decode()), 
                    'comments': {}
                }
                
                _dict['data'].update({"upvotes": 0, "downvotes": 0})
            return True
        result = add_comment(_data, _dict['comments'], _is_vote)
        if result:
            return True
    return False

In [354]:
posts = {}
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    category = schema_items[0]
    post_hash = schema_items[1]
    posts[post_hash] = {'data': {"upvotes": 0, "downvotes": 0}, 'comments': {}}
    
    for content in contents:
        data = bytes.fromhex(content['data'][2:]).decode().split(',')
        # votes had numeric numbers instead of hashes
        is_vote = data[3].strip('-').isnumeric() 
        if is_vote:
            data[3] = int(data[3])
            
        if data[2] == '': 
            # empty means its post data
            posts[post_hash]['data'].update(json.loads(client.cat(data[3]).decode()))
        elif data[2] == post_hash: 
            # post_hash means its comment
            if is_vote:
                if data[3] < 0:
                    # downvote
                    posts[post_hash]['data']['downvotes'] += data[3]
                else:
                    # upvote
                    posts[post_hash]['data']['upvotes'] += data[3]
            else:
                posts[post_hash]['comments'][data[3]] = {
                    'data': json.loads(client.cat(data[3]).decode()), 
                    'comments': {}
                }
                posts[post_hash]['comments'][data[3]]['data'].update({"upvotes": 0, "downvotes": 0})
        else: 
            # neither means its comment of comment
            result = add_comment(data[3], posts[post_hash]['comments'], is_vote)
            if not result:
                print('couldnt find', data[2], json.loads(client.cat(data[3]).decode()))

In [355]:
from collections import OrderedDict
o = OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True))

In [357]:
posts_list = []
for k, v in OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True)).items():
    posts_list.append({k:v})